In [ ]:
#add mbes_sim to search path
import sys
sys.path.insert(0, "../src/")

In [ ]:
from tqdm.auto import tqdm
from copy import deepcopy
import time
import math

from IPython.display import clear_output

import mbes_sim.simulationfunctions as SIMFUN

In [ ]:
# setups base comparison
#from collections import set

def addSimSetup(simSetups,options):
    simsetup = deepcopy(BaseSetup)
    for kopt,vopt in options.items():
        for k,v in vopt.items():
            simsetup[k] = v
                                
    sname_blockAvg = ''
    sname = ''
    for k,v in simsetup.items():
        sname +=k+'['+str(v)+']'
                                    
        if not 'blockAvg' in k:
            sname_blockAvg +=k+'['+str(v)+']'
        else:
            sname_blockAvg += k+'['+str(True)+']'
            
    if sname_blockAvg in simSetups.keys():
       return       
    simSetups[sname] = simsetup
    


In [ ]:

#create base setup

setup = SIMFUN.SimulationSetup(
    prefix = 'sim_batch',
    blockAvg = True,
    windowType           = SIMFUN.t_Window.Exponential,
    motionDataPath = '../test_data/m143_l0154_motion.csv',
    surveyType           = SIMFUN.t_Survey.IdealMotion,
    voxelsize            = 3,
    surveyspeedKnots     = 3,
    swathdistance        = 0.8,
    layerDepths          = [],
    layerSizes           = [],
    bubbleType           = SIMFUN.t_Bubbles.SingleBubble,
    exagHPR              = 1,
    BaseDirectory        = 'simresults',
    
    load_previous_simresults = False,    
    verbose = True)

BaseSetup = deepcopy(setup.SimSetup)

In [ ]:
# add batch options for single bubble, voxelsize 3 ideal motion

BaseOptions={'blockAvg' : True}

windowOptions=[]
windowOptions.append({'windowType' : SIMFUN.t_Window.Box})
windowOptions.append({'windowType' : SIMFUN.t_Window.Hann})
windowOptions.append({'windowType' : SIMFUN.t_Window.Exponential})

voxelOptions=[]
voxelOptions.append({'voxelsize' : 1 })
voxelOptions.append({'voxelsize' : 1.5 })
voxelOptions.append({'voxelsize' : 3 })

navOptions=[]
navOptions.append({'surveyType' : SIMFUN.t_Survey.IdealMotion})


options = {}
simSetups = {}

options['base'] = BaseOptions    
for wo in windowOptions:
    options['wo'] = wo    
    for vo in voxelOptions:
        options['vo'] = vo                    
        for no in navOptions:
            options['no'] = no
    
            addSimSetup(simSetups,options)
                            
len(simSetups)

In [ ]:
# add batch options for single bubble, voxelsize 1-3 real motion

BaseOptions={'blockAvg' : False}

windowOptions=[]
#windowOptions.append({'windowType' : SIMFUN.t_Window.Box})
#windowOptions.append({'windowType' : SIMFUN.t_Window.Hann})
windowOptions.append({'windowType' : SIMFUN.t_Window.Exponential})

voxelOptions=[]
voxelOptions.append({'voxelsize' : 1 })
#voxelOptions.append({'voxelsize' : 1.5 })
voxelOptions.append({'voxelsize' : 3 })

navOptions=[]
navOptions.append({'surveyType' : SIMFUN.t_Survey.IdealMotion})
# navOptions.append({'surveyType' : SIMFUN.t_Survey.RealMotion,
#                   'exagHPR' : 1})
navOptions.append({'surveyType' : SIMFUN.t_Survey.RealMotion,
                  'exagHPR' : 3})


options = {}
#simSetups = {}

options['base'] = BaseOptions    
for wo in windowOptions:
    options['wo'] = wo    
    for vo in voxelOptions:
        options['vo'] = vo                    
        for no in navOptions:
            options['no'] = no
    
            addSimSetup(simSetups,options)
                            
len(simSetups)

In [ ]:
# add batch options for bubble streams
BaseOptions={'blockAvg' : False,
             'layerDepths' : [42,48,60,108,42,48,60,108],
             'layerSizes'  : [12,12,12, 12,24,24,24,24],
             'bubbleType'  : SIMFUN.t_Bubbles.BubbleStream    }

windowOptions=[]
#windowOptions.append({'windowType' : SIMFUN.t_Window.Box})
#windowOptions.append({'windowType' : SIMFUN.t_Window.Hann})
windowOptions.append({'windowType' : SIMFUN.t_Window.Exponential})

voxelOptions=[]
voxelOptions.append({'voxelsize' : 1 })
#voxelOptions.append({'voxelsize' : 1.5 })
#voxelOptions.append({'voxelsize' : 3 })

navOptions=[]
#navOptions.append({'surveyType' : SIMFUN.t_Survey.IdealMotion})
navOptions.append({'surveyType' : SIMFUN.t_Survey.RealMotion,
                  'exagHPR' : 1})
#navOptions.append({'surveyType' : SIMFUN.t_Survey.RealMotion,
#                  'exagHPR' : 3})

options = {}
#simSetups = {}

options['base'] = BaseOptions    
for wo in windowOptions:
    options['wo'] = wo    
    for vo in voxelOptions:
        options['vo'] = vo                    
        for no in navOptions:
            options['no'] = no
    
            addSimSetup(simSetups,options)
                            
len(simSetups)

In [ ]:
#print batch setups
if True:
    for i,(_,ss) in enumerate(simSetups.items()):
        print('-- Setup [{}] --'.format(i+1))
        if i == 0:
            bss = deepcopy(ss)
            for k,v in ss.items():
                print(k,'=',v)
        else:
            for k,v in ss.items():
                if bss[k] != v:
                    print(k,'=',v)

        print()

    
print(len(simSetups))

In [ ]:
print(len(simSetups))
print('started:',time.time())

In [ ]:
TotalSims =1200 # total number of simulations per setup
StartMainRun = 2
MainRuns = 20  # number of simulation steps (each simulation step will simulate TotalSims/MainRuns for each setup)

parallel_sims = 2

try:
    MainProgress.close()
    del MainProgress
    SetupProgress.close()
    del SetupProgress
except:
    pass

MainProgress  = tqdm(total = MainRuns, desc='MainProgress',position = 1)
SetupProgress = tqdm(total = len(simSetups), desc='SetupProgress',position = 2)

In [ ]:

MainProgress.reset(total=MainRuns)

localSims = math.ceil(TotalSims/MainRuns)
currentResults = 0
    
for subsimrun in range(StartMainRun,MainRuns):
    SetupProgress.reset(total=len(simSetups))
    for sn,(_,simSetup) in enumerate(simSetups.items()):
        #importlib.reload(tqdmModule)
        clear_output()
        print(' --- SubSimRun: {}/{} ---'.format(subsimrun+1,MainRuns))
        print('simSetup: {}/{}'.format(sn+1,len(simSetups)))
        for k,v in simSetup.items():
            print(k,'=',v)
        print()
        print('Loading previous results ...')
        simreturns = setup.get_previous_simreturns(simSetup,setup.BaseDirectory)
        
        if not setup.AddPlots:
            print('deleting plots')
            for k in simreturns.keys():
                if 'resample' in k:
                    del simreturns[k]
                    print('1',k)
            for k in simreturns.keys():
                if 'resample' in k:
                    del simreturns[k]
                    print('1',k)
            
        currentTotalSims = (localSims) * (subsimrun+1)
        print('currentTotalSims:            ',currentTotalSims)
        print('len(setup.SimulationResults):',len(simreturns))
            
        if len(simreturns) < (localSims) * (subsimrun+1):
            localTotalSims = deepcopy(localSims)
            if len(simreturns) + localTotalSims > currentTotalSims:
                localTotalSims = currentTotalSims - len(simreturns)
                
            parasims = parallel_sims
            if parasims > localTotalSims:
                parasims = localTotalSims
            
            print('--- initializing setup ---')
            
            
            setup.call_setups(simSetup,verbose = True,load_previous_simresults = False)
            setup.SimulationResults = simreturns
            
            print('--- Simulate ---')
            setup.simulate(totalSimulations    = localTotalSims,
                           parallelSimulations = parasims)
        else:
            print('skipping ...')
            time.sleep(1)
        
        
        SetupProgress.update()
        MainProgress.update(1/len(SetupProgress))